## Carregando Dados e Bibliotecas necessárias.

In [1]:
import pandas as pd
import numpy as np

df = pd.read_csv('../raw/airbnb-dataset.csv', low_memory=False)
print("Dataset carregado com sucesso!")
df.head()

Dataset carregado com sucesso!


,id,NAME,host id,host_identity_verified,host name,neighbourhood group,neighbourhood,lat,long,country,...,service fee,minimum nights,number of reviews,last review,reviews per month,review rate number,calculated host listings count,availability 365,house_rules,license
0,1001254,Clean & quiet apt home by the park,80014485718,unconfirmed,Madaline,Brooklyn,Kensington,40.64749,-73.97237,United States,...,$193,10.0,9.0,10/19/2021,0.21,4.0,6.0,286.0,Clean up and treat the home the way you'd like...,NaN
1,1002102,Skylit Midtown Castle,52335172823,verified,Jenna,Manhattan,Midtown,40.75362,-73.98377,United States,...,$28,30.0,45.0,5/21/2022,0.38,4.0,2.0,228.0,Pet friendly but please confirm with me if the...,NaN
2,1002403,THE VILLAGE OF HARLEM....NEW YORK !,78829239556,NaN,Elise,Manhattan,Harlem,40.80902,-73.94190,United States,...,$124,3.0,0.0,NaN,NaN,5.0,1.0,352.0,"I encourage you to use my kitchen, cooking and...",NaN
3,1002755,NaN,85098326012,unconfirmed,Garry,Brooklyn,Clinton Hill,40.68514,-73.95976,United States,...,$74,30.0,270.0,7/5/2019,4.64,4.0,1.0,322.0,NaN,NaN
4,1003689,Entire Apt: Spacious Studio/Loft by central park,92037596077,verified,Lyndon,Manhattan,East Harlem,40.79851,-73.94399,United States,...,$41,10.0,9.0,11/19/2018,0.10,3.0,1.0,289.0,"Please no smoking in the house, porch or on th...",NaN


# Tratamento dos Dados.

## Padronização dos Nomes das Colunas.

In [2]:
df.rename(
    columns={col: col.lower().replace(' ', '_') for col in df.columns},
    inplace=True
)
print(df.columns)

Index(['id', 'name', 'host_id', 'host_identity_verified', 'host_name',
       'neighbourhood_group', 'neighbourhood', 'lat', 'long', 'country',
       'country_code', 'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'price', 'service_fee', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'review_rate_number', 'calculated_host_listings_count',
       'availability_365', 'house_rules', 'license'],
      dtype='object')


## Remoção de Colunas Desnecessárias

Como quase todas as tuplas de **license** estavam como nulas, optamos por não trabalhar com essa coluna. Além disso, optamos por remover as colunas de **Country** e **Country_code** visto que sabemos que todas se enquadram no Estados Unidos e possuem o códido do país como "US".

In [3]:
cols_to_drop = ['country', 'country_code', 'license']
df.drop(columns=cols_to_drop, inplace=True)

for col in cols_to_drop:
    if col not in df.columns:
        print(f"Coluna {col} deletada!")

print("Colunas: ")
print(df.columns)

Coluna country deletada!
Coluna country_code deletada!
Coluna license deletada!
Colunas: 
Index(['id', 'name', 'host_id', 'host_identity_verified', 'host_name',
       'neighbourhood_group', 'neighbourhood', 'lat', 'long',
       'instant_bookable', 'cancellation_policy', 'room_type',
       'construction_year', 'price', 'service_fee', 'minimum_nights',
       'number_of_reviews', 'last_review', 'reviews_per_month',
       'review_rate_number', 'calculated_host_listings_count',
       'availability_365', 'house_rules'],
      dtype='object')


## Correções dos Tipos de Dados.

1) **price** e **service_fee** possuem o caracter especial "$" e estão como String. Com isso, iremos altera-las para o tipo númerico (Float)

In [4]:
money_columns = ['price', 'service_fee']

print("Tipos de dados antes da correção:")
print(df[money_columns].dtypes)

for col in money_columns:
    df[col] = df[col].str.replace('$', '', regex=False).str.replace(',', '', regex=False).str.strip().astype(float)

print("Tipos de dados corrigidos:")
print(df[money_columns].dtypes)

Tipos de dados antes da correção:
price          object
service_fee    object
dtype: object
Tipos de dados corrigidos:
price          float64
service_fee    float64
dtype: object


2) **construction_year**, **availability_365** e **calculated_host_listings_count**: Não faria sentido estar sendo guardado em Float já que era o ano de construção. Ou seja, nunca viria um número decimal. O mesmo vale para availability_365 e para calculated_host_listings_count.

In [5]:
floats_to_ints = ['availability_365', 'construction_year','calculated_host_listings_count']

for col in floats_to_ints:
    print(f"Tipo de dado da chave {col} antes da correção: {df[col].dtype}")


    df[col] = df[col].astype('Int64')

    print(f"Tipo de dado da chave {col} após a correção: {df[col].dtype}")

Tipo de dado da chave availability_365 antes da correção: float64
Tipo de dado da chave availability_365 após a correção: Int64
Tipo de dado da chave construction_year antes da correção: float64
Tipo de dado da chave construction_year após a correção: Int64
Tipo de dado da chave calculated_host_listings_count antes da correção: float64
Tipo de dado da chave calculated_host_listings_count após a correção: Int64


3) **host_identity_verified**: A coluna host_identity_verified pode ser transformada em booleano (True/False), o que é mais eficiente e semanticamente correto.

In [6]:
df['host_identity_verified'] = df['host_identity_verified'].astype(str).str.lower().str.strip()

to_replace = {
    'verified': True,
    'unconfirmed': False
}

df['host_identity_verified'] = df['host_identity_verified'].replace(to_replace).astype(bool)


print("Tipo de dado da coluna APÓS o tratamento:")
print(df['host_identity_verified'].dtype)
print("\nValores únicos na coluna APÓS o tratamento:")
print(df['host_identity_verified'].unique())
print("\nContagem de valores na coluna APÓS o tratamento:")
print(df['host_identity_verified'].value_counts(dropna=False))

Tipo de dado da coluna APÓS o tratamento:
bool

Valores únicos na coluna APÓS o tratamento:
[False  True]

Contagem de valores na coluna APÓS o tratamento:
host_identity_verified
True     51399
False    51200
Name: count, dtype: int64


## Correção de Inconsistência nos Dados.

### Correção nos erros de digitação no nome dos bairros que apresentavam "brookln" e "manhatan"

In [7]:

df['neighbourhood_group'] = df['neighbourhood_group'].replace({
    'brookln': 'Brooklyn',
    'manhatan': 'Manhattan'
})


print(df['neighbourhood_group'].unique())

['Brooklyn' 'Manhattan' 'Queens' nan 'Staten Island' 'Bronx']


### Tratamento de Valores Ausentes.


1) Remoção de Anúncios sem preço.

In [8]:

df.dropna(subset=['price', 'service_fee'], inplace=True)

print(f"Valores nulos em 'price' após remoção: {df['price'].isnull().sum()}")

Valores nulos em 'price' após remoção: 0


2) Criação de Coluna booleana para house_rules: Como metade dos valores é nulo, iremos criar uma nova coluna para indicar se essa "casa" possui ou não regras definidas.

In [9]:

df['has_house_rules'] = df['house_rules'].notna()

# Podemos agora remover a coluna original se o conteúdo de texto não for usado
# df_silver.drop(columns=['house_rules'], inplace=True)


print(df['has_house_rules'].value_counts())

has_house_rules
False    51853
True     50260
Name: count, dtype: int64


3) Preenchimento dos poucos anúncios sem nome (ou sem nome de host) com "Sem nome informado"

In [10]:
for col in ['name', 'host_name']:
    df[col] = df[col].fillna('Sem nome informado')

4) Remoção dos demais **nans**

In [11]:
nans_to_drop = [
    'neighbourhood', 
    'neighbourhood_group', 
    'lat', 
    'long',
    'host_identity_verified',
    'minimum_nights',
]

df.dropna(subset=nans_to_drop, inplace=True)

## Tratamentos de Valores invalidados.

Foi identificado alguns valores negativos na coluna **availability_365** que não fazem sentido com o escopo.

In [12]:
df = df[df['availability_365'] >= 0]
print(df['availability_365'].min())

0


## Dicionario - Silver

In [13]:
print("--- Amostra Aleatória de 20 Linhas do DataFrame 'df_silver' ---")

# O .sample(20) pega 20 linhas aleatórias do DataFrame.
# O .reset_index(drop=True) é para a visualização ficar mais limpa, sem o índice antigo.
display(df.sample(20).reset_index(drop=True))


print("\n\n--- Resumo das Informações (Info) ---")
df.info()

--- Amostra Aleatória de 20 Linhas do DataFrame 'df_silver' ---


,id,name,host_id,host_identity_verified,host_name,neighbourhood_group,neighbourhood,lat,long,instant_bookable,...,service_fee,minimum_nights,number_of_reviews,last_review,reviews_per_month,review_rate_number,calculated_host_listings_count,availability_365,house_rules,has_house_rules
0,32949682,NOTRE-DAME Room in House - SELF CHECK IN,78355133172,False,Frederick And Jessy,Queens,Howard Beach,40.653576,-73.835815,False,...,84.0,4.0,3.0,1/7/2022,0.73,3.0,4,244,NaN,False
1,20137979,Private Suite,70999640624,False,Dennis,Queens,Astoria,40.757590,-73.917030,True,...,127.0,5.0,1.0,9/29/2018,0.11,3.0,1,69,"Please, no smoking indoors. Do not climb onto ...",True
2,30274341,Large duplex steps from Central Park!,97659079356,False,Marisa,Manhattan,Upper West Side,40.782980,-73.978710,False,...,97.0,2.0,1.0,2/20/2022,1.00,4.0,1,71,NaN,False
3,40919923,Amazing Riverside Park Studio,3084906237,True,Abby,Manhattan,Upper West Side,40.781290,-73.985720,True,...,88.0,2.0,7.0,10/22/2015,0.15,4.0,2,0,NaN,False
4,17880176,2 Bedroom Brooklyn Apt Minutes from NYC Landma...,77676791676,True,Bashiri,Brooklyn,Bedford-Stuyvesant,40.684500,-73.958040,True,...,136.0,2.0,44.0,7/1/2019,2.72,4.0,1,285,NaN,False
5,25154520,Modern Private Bedroom in Upper West Side,81231233379,True,Eyal,Manhattan,Upper West Side,40.800060,-73.960950,False,...,134.0,30.0,0.0,NaN,NaN,2.0,32,37,Please remember that this is a residential bui...,True
6,49714748,Bedroom in SoHo/Nolita,11717032408,False,Alessandra,Manhattan,Nolita,40.721270,-73.995260,True,...,182.0,4.0,2.0,5/7/2019,0.32,2.0,1,0,NaN,False
7,57198966,MONTHLY PRICE- Perfect for one person in Astoria,57893904364,False,Luiza,Queens,Astoria,40.758320,-73.914480,True,...,168.0,15.0,23.0,5/31/2019,0.83,5.0,2,31,- The building and its premises and grounds ar...,True
8,48542768,Incredible 3 BR Union Square Loft,51392537353,True,Anna,Manhattan,Gramercy,40.736270,-73.989860,False,...,43.0,7.0,0.0,NaN,NaN,2.0,1,209,No Smoking No Pets,True
9,27539351,"Private room w/ Loft Bed, Desk, Quick Ride to ...",60093249298,False,Nikolas,Brooklyn,Bushwick,40.690570,-73.906710,False,...,211.0,2.0,1.0,6/30/2019,1.00,4.0,1,15,Please remember that this is a residential bui...,True




--- Resumo das Informações (Info) ---
<class 'pandas.core.frame.DataFrame'>
Index: 100817 entries, 0 to 102598
Data columns (total 24 columns):
 #   Column                          Non-Null Count   Dtype  
---  ------                          --------------   -----  
 0   id                              100817 non-null  int64  
 1   name                            100817 non-null  object 
 2   host_id                         100817 non-null  int64  
 3   host_identity_verified          100817 non-null  bool   
 4   host_name                       100817 non-null  object 
 5   neighbourhood_group             100817 non-null  object 
 6   neighbourhood                   100817 non-null  object 
 7   lat                             100817 non-null  float64
 8   long                            100817 non-null  float64
 9   instant_bookable                100739 non-null  object 
 10  cancellation_policy             100756 non-null  object 
 11  room_type                       100817 non-

## Salvando Dataset

In [14]:

df.to_csv('airbnb-dataset-silver.csv', index=False)

print("Dataset da camada Silver salvo com sucesso!")

Dataset da camada Silver salvo com sucesso!


In [15]:
df['availability_365'].dtype

Int64Dtype()